In [5]:
import numpy as np

In [36]:
class Layer:

    def relu(self,X):

        
        return np.maximum(0,X),X
    
    def linear(self,X):
        return X,X
    
    def sigmoid(self,X):
        return 1/(1 + np.exp(-X)),X

    def __init__(self,unit_num,act_func,input_size) -> None:
        
        self.input_size = input_size
        self.unit_num = unit_num
        self.act_func = act_func
        self.weights = np.random.rand(unit_num,input_size)*0.2
        self.bias = np.zeros((unit_num,1))

        



    def forward_prop(self,inp):
        if self.act_func == "sigmoid":
            return self.sigmoid(np.dot(self.weights,inp) + self.bias)
        
        elif self.act_func == "relu":
            return self.relu(np.dot(self.weights,inp) + self.bias)
        

    def __str__(self) -> str:
        return f"Layer with {self.unit_num} units and {self.act_func} activation function."
        
        


        

In [37]:
class NN:

    def __init__(self) -> None:
        self.layers = []
        return None
    def add_layer(self,input_size,activation_function,layer_size):

        if (len(self.layers) == 0) or (input_size == self.layers[-1].unit_num):
            new_layer = Layer(unit_num=layer_size,act_func = activation_function,input_size=input_size)
            self.layers.append(new_layer)
            print("Added a layer!")
        else:
            raise Exception("Input Size don't match with the last layer's output size!")



        return None
    
    def __str__(self) -> str:

        print(f"Neural Network with {len(self.layers)} layers.")
        for layer in self.layers:
            print(layer)
        
        return ""


    def forward_propagation(self,X):
        output = [self.layers[0].forward_prop(X)[0]]
        print(output[0])



        for i in range(1,len(self.layers)):
            output.append(self.layers[i].forward_prop(output[i-1])[0])
            print(f"Output of layer {i+1}:\n{output[i]}")


        
    
    #derivative of sigmoid is f(x)*(1-f(x))

    def sigmoid_back(self,dA,cache):
        Z = cache
        s= 1/(1+np.exp(-Z))

        dZ = dA * s * (1-s)

        return dZ
    

    def relu_back(seld,dA,cache):
        

        Z = cache

        dZ = np.copy(dA)

        dZ[Z <= 0] = 0

        return dZ
    


    def compute_cost(self,cost_func,Y,yhat,epsilon):

        if cost_func =="crossentropy":
            
            cost = np.sum((1-Y) * (-np.log(1-yhat + epsilon)) - Y * (np.log(yhat + epsilon)))/len(Y)
            print(cost)
            return cost
        

    def linear_backward(self,dZ,cache):

        A_prev, W, b = cache

        m = len(A_prev)

        dW = (1/m) * np.dot(dZ,A_prev.T)

        db = (1/m) * np.sum(dZ,axis=1,keepdims=True)

        dA_prev = np.dot(W.T,dZ)

        return dA_prev, db, dW
    
    def linear_activation_backward(self,dA,activation,cache):

        activation_cache, linear_cache = cache 

        if activation == "sigmoid":
            dZ = self.sigmoid_back(dA,activation_cache)

            dA_prev, dW, db = self.linear_backward(dZ,linear_cache)

        elif activation =="relu":

            dZ = self.relu_back(dA,activation_cache)

            dA_prev, dW, db = self.linear_backward(dZ,linear_cache)
        

        return dA_prev, dW, db
    


    def backprop(self,AL,Y,caches):


        '''
        AL: output of forward propagation
        Y : true values
        caches:every cache of layers
        
        
        '''

        grads = {}

        L = len(self.layers)

        m = len(Y)

        dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))


        current_cache = caches[L-1]

        grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = self.linear_activation_backward(dAL, current_cache, "sigmoid")


        for l in reversed(range(L-1)):

            current_cache = caches[l]
            dA_prev_temp, dW_temp, db_temp = self.linear_activation_backward(grads["dA"+str(l+1)], current_cache, "relu")
            grads["dA" + str(l)] = dA_prev_temp
            grads["dW" + str(l + 1)] = dW_temp
            grads["db" + str(l + 1)] = db_temp

        return grads
    

    
    def update_params(self,parameters,grads, alpha):

        
        '''
        alpha: learning rate
        '''
        L = len(parameters) // 2 

    # Update rule for each parameter
        for l in range(L):
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - alpha*grads["dW" + str(l+1)]
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - alpha*grads["db" + str(l+1)]

        return parameters















    






    


In [54]:
X = np.array([[1],[2],[3]])


deneme = NN()
deneme.add_layer(3,"sigmoid",5)
deneme.add_layer(5,"relu",10)
deneme.add_layer(10,"sigmoid",1)

Added a layer!
Added a layer!
Added a layer!


In [55]:
deneme.forward_propagation(X)

[[0.58851021]
 [0.64035473]
 [0.65494873]
 [0.69048071]
 [0.69515386]]
Output of layer 2:
[[0.71955601]
 [0.08221952]
 [0.32708679]
 [0.33770135]
 [0.05573813]
 [0.65604154]
 [0.08134684]
 [0.88129575]
 [0.55005572]
 [0.36025082]]
Output of layer 3:
[[0.50207877]]
